In [ ]:
import torch 
import os 
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

%cd ../

In [ ]:
from models.pytorchvideo_models import WalkVideoClassificationLightningModule
from pytorch_lightning import Trainer

In [ ]:
from main import get_parameters

opt, unknown = get_parameters()
opt.num_workers = 8
opt.batch_size = 64
opt.gpu_num = 0
opt.model = "csn"
opt.model_depth = 50


In [ ]:
model = WalkVideoClassificationLightningModule(opt)
# model = model.load_from_checkpoint("/workspace/Walk_Video_PyTorch/logs/resnet/0603/checkpoints/epoch=99-step=1800.ckpt")
model.eval()

checkpoint = torch.load("/workspace/Walk_Video_PyTorch/logs/csn/0719_csn_depth50/checkpoints/epoch=40-step=8200.ckpt")

model.load_state_dict(checkpoint["state_dict"])

In [ ]:
from dataloader.data_loader import WalkDataModule
from pytorch_lightning import loggers as pl_loggers 

# load test dataset 
module = WalkDataModule(opt)
module.setup()
test_data = module.test_dataloader()

# for the tensorboard
tb_logger = pl_loggers.TensorBoardLogger(save_dir="/workspace/Walk_Video_PyTorch/project/tests/logs", name=opt.model, version=opt.version)


trainer = Trainer(
    accelerator="auto",
    devices=1,
    gpus=opt.gpu_num,
    logger=tb_logger,
    max_epochs=-1,
)

trainer.test(dataloaders=module, model=model)


In [ ]:
input_data = next(iter(test_data))


In [ ]:
input_data['video'].shape

input_data["video_name"]

In [ ]:
input_data['label']

In [ ]:
preds = model(input_data['video'])

In [ ]:
post_act = torch.nn.Softmax(dim=1)
preds = post_act(preds)
pred_classes = preds.topk(k=1).indices

In [ ]:
pred_classes

In [ ]:
classname = {}

classname[0] = 'asd'
classname[1] = 'asd_not'

In [ ]:
real_calss = []

for i in input_data['label'].tolist():
    real_calss.append(classname[i])

In [ ]:
real_calss

In [ ]:
# pred_class_names = []
# for num in range(opt._BATCH_SIZE):
#     for i in pred_classes[i]:
#         pred_class_names.append(classname[int(i)])


pred_class_names = [classname[int(i)] for i in pred_classes]
print("Predicted labels: %s" % ", ".join(pred_class_names))
print("real label: %s" % ",".join(real_calss))

In [ ]:
pred_class_names == real_calss

In [ ]:
result = []

for i in range(len(real_calss)):
    if pred_class_names[i] == real_calss[i]:
        result.append("true")
    else:
        result.append("false")

result